In [11]:
#Processing evcc block ack
import csv
import numpy as np

with open('csv/DataLink/no_inter/no_inter_evcc_blockack.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    startint_seq_num_block_ack = []
    timelist_block_ack = []
    
    for row in rows:
        startint_seq_num_block_ack.append(int(row['Starting Sequence Number']))
        timelist_block_ack.append(float(row['Time']))

In [12]:
#for debug
if(len(startint_seq_num_block_ack) != len(timelist_block_ack)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_block_ack))

1281


In [13]:
#Processing evcc send
#Append missing SECC Discovery Req. time = 51.312512
with open('csv/DataLink/no_inter/no_inter_evcc_send_append_missing_seccdisreq.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    timelist_evcc_send = []
    seq_num_evcc_send = []
    data_length_evcc_send = []
    info_evcc_send = []
    frame_num_evcc_send = []
    
    for row in rows:
        timelist_evcc_send.append(float(row['Time']))
        seq_num_evcc_send.append(int(row['Sequence number']))
        data_length_evcc_send.append(int(row['Data Length']))
        info_evcc_send.append(str(row['Info']))
        frame_num_evcc_send.append(int(row['No.']))

In [14]:
#for debug
if(len(timelist_evcc_send) != len(seq_num_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(data_length_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(info_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(frame_num_evcc_send)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_evcc_send))

1287


In [15]:
timelist_evcc_send_split = []
seq_num_evcc_send_split = []
data_length_evcc_send_split = []
info_evcc_send_split = []
frame_num_evcc_send_split = []

round_start_point = 0
round_end_point = 0

sdpreq_index_list = []
#Fetch first packet index of data length = 74 (SDP request) in each round
for i in range(0, len(data_length_evcc_send)):
    if(i == 0 and data_length_evcc_send[i] == 74):
        sdpreq_index_list.append(i)
    if(i > 0 and data_length_evcc_send[i] == 74 and data_length_evcc_send[i-1] != 74):
        sdpreq_index_list.append(i)
sdpreq_index_list.append(len(data_length_evcc_send)) #Add the end of list index

for i in range(0, len(sdpreq_index_list)-1):
    timelist_evcc_send_split.append([timelist_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    seq_num_evcc_send_split.append([seq_num_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    data_length_evcc_send_split.append([data_length_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    info_evcc_send_split.append([info_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    frame_num_evcc_send_split.append([frame_num_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])

In [16]:
#for debug
if(len(timelist_evcc_send_split) != len(seq_num_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(data_length_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(info_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(frame_num_evcc_send_split)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_evcc_send_split)) #Output # of testing rounds

20


In [17]:
#for debug, this is ideally the time of each testing round
for i in range(1, len(timelist_evcc_send_split)):
    print(timelist_evcc_send_split[i][0]-timelist_evcc_send_split[i-1][0])

19.91932
19.404170999999998
19.174493999999996
19.13987300000001
19.352576
19.148584
19.251359000000008
19.251021999999978
19.251002
19.148754000000025
19.14927
19.148322999999976
19.251022000000034
19.25108899999998
19.25129800000002
19.148646999999983
19.251169000000004
19.046310000000005
19.046175000000005


In [ ]:
#Set evcc time anchor in each round per stage
evcc_send_time_anchor = []
#data_length_in_each_stage = [74, 96, 189, 173, 173, 173, 185, 461, 189, 173]
wierd_packet = {}
packet_loss_testing_round_6_flag = False
packet_loss_testing_round_17_flag = False
packet_loss_testing_round_19_flag = False

def printinfo(stageNum, frameNum, info_string):
    print("-----------------------------------------")
    print("Stage " + str(stageNum) + " finish. No. " + str(frameNum) + ", Info " + str(info_string))

def print_time_difference(evcc_send_time_anchor_each_round):
    print("Time difference: " + str(evcc_send_time_anchor_each_round[len(evcc_send_time_anchor_each_round)-1]-
                          evcc_send_time_anchor_each_round[len(evcc_send_time_anchor_each_round)-2]))
    
def checkstage_num(stageNum, valid_stageNum, info_string, time_string, datalen, seqnum, frameNum):
    if(stageNum == valid_stageNum):
        check_wield_packet_list(frameNum)
        return True
    else:
        #debug
        print("\nFalse")
        print("Stage Num = " + str(stageNum))
        print("Valid Stage Num = " + str(valid_stageNum))
        print("What is this packet? Data Length = " + str(datalen) + " No. " + str(frameNum))
        print("Time " + str(time_string) + " #Seq " + str(seqnum) + " Info " + str(info_string) + "\n")
        wierd_packet[frameNum] = [datalen, stageNum]
        return False

def check_wield_packet_list(frameNum):
    if(wierd_packet.get(frameNum) != None):
        del wierd_packet[frameNum]
        print("\nNo. " + str(frameNum) + " has been found. Delete from wierd list.\n")
    
for i in range(0, len(data_length_evcc_send_split)):
#for i in [18]:
    print("\n-------- Test Round " + str(i+1) + " --------")
    #Stage 1
    evcc_send_time_anchor_each_round = [timelist_evcc_send_split[i][0]]
    stageNum = 1
    printinfo(stageNum, frame_num_evcc_send_split[i][0], info_evcc_send_split[i][0])
    stageNum = stageNum + 1
    
    for j in range(1, len(data_length_evcc_send_split[i])):
        #Stage 2
        #Deal with packet loss......
        if(i == 5 and packet_loss_testing_round_6_flag == False):
            evcc_send_time_anchor_each_round.append(109)
            printinfo(stageNum, 13136, "Packet Loss")
            print_time_difference(evcc_send_time_anchor_each_round)
            stageNum = stageNum + 1
            packet_loss_testing_round_6_flag = True
        
        
        if(data_length_evcc_send_split[i][j] == 96 and data_length_evcc_send_split[i][j-1] != 96):
            if(checkstage_num(stageNum, 2, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 3
        if(data_length_evcc_send_split[i][j] == 189 and data_length_evcc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 3, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 4 
        if(data_length_evcc_send_split[i][j] == 173 and data_length_evcc_send_split[i][j-1] != 173):
            
            #if(info_evcc_send_split[i][j][-2:] == 'TC'):
            if(checkstage_num(stageNum, 4, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
            
        #Stage 5, tricky
        if(data_length_evcc_send_split[i][j] == 173):    
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 5, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
            
            else: #info_evcc_send_split[i][j][-2:] == 'FC'
                #Backward checking 2 packets
                isTCfound = False
                for k in range(1, 3):
                    if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                        isTCfound = True
                        break
                if(info_evcc_send_split[i][j][-5:] == 'R.F.C'):
                    continue
                
                if(not isTCfound):
                    if(checkstage_num(stageNum, 5, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        print("Stage " + str(stageNum) +
                            ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                            ", Data length " + str(data_length_evcc_send_split[i][j]) +
                            ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                        
                        #Deal with packet loss......
                        if(i == 16 and packet_loss_testing_round_17_flag == False):
                            evcc_send_time_anchor_each_round.append(320.953)
                            printinfo(stageNum, 38022, "Packet Loss")
                            print_time_difference(evcc_send_time_anchor_each_round)
                            stageNum = stageNum + 1
                            packet_loss_testing_round_17_flag = True

                        continue
                
        #Stage 6, tricky
        if(data_length_evcc_send_split[i][j] == 173):
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 6, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else: #info_evcc_send_split[i][j][-2:] == 'FC'
                #Backward checking 2 packets
                isTCfound = False
                for k in range(1, 3):
                    if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                        isTCfound = True
                        break

                if(not isTCfound):
                    if(checkstage_num(stageNum, 6, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        print("Stage " + str(stageNum) +
                            ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                            ", Data length " + str(data_length_evcc_send_split[i][j]) +
                            ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1

                        continue
        
        #Stage 7
        if(data_length_evcc_send_split[i][j] == 185 and data_length_evcc_send_split[i][j-1] != 185):
            if(checkstage_num(stageNum, 7, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                #Deal with packet loss......
                if(i == 18 and packet_loss_testing_round_19_flag == False):
                    evcc_send_time_anchor_each_round.append(359.320)
                    printinfo(stageNum, 42319, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_19_flag = True
                
                continue
                
        #Stage 8
        if(data_length_evcc_send_split[i][j] == 461 and data_length_evcc_send_split[i][j-1] != 461):
            if(checkstage_num(stageNum, 8, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
        #Stage 9
        if(data_length_evcc_send_split[i][j] == 189 and data_length_evcc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 9, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 10
        if(data_length_evcc_send_split[i][j] == 173 and data_length_evcc_send_split[i][j-1] != 173):
            
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 10, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
            else: #info_evcc_send_split[i][j][-2:] == 'FC'
                #Backward checking 2 packets
                isTCfound = False
                for k in range(1, 3):
                    if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                        isTCfound = True
                        break

                if(not isTCfound):
                    if(checkstage_num(stageNum, 10, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        print("Stage " + str(stageNum) +
                            ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                            ", Data length " + str(data_length_evcc_send_split[i][j]) +
                            ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                
        if(stageNum == 11): break
        
    evcc_send_time_anchor.append(evcc_send_time_anchor_each_round)

In [46]:
#for debug
print(str(len(evcc_send_time_anchor)) + '\n')
for i in range(0, len(evcc_send_time_anchor)):
    print(len(evcc_send_time_anchor[i]))

20

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [48]:
wierd_packet

{1340: [173, 3],
 1575: [185, 8],
 4099: [173, 3],
 4282: [185, 8],
 6502: [173, 3],
 8897: [185, 8],
 11070: [173, 3],
 11163: [173, 5],
 11262: [185, 8],
 13260: [173, 3],
 15524: [173, 3],
 19788: [173, 3],
 22872: [173, 3],
 25448: [173, 3],
 27550: [173, 3],
 29649: [173, 3],
 31602: [173, 3],
 33436: [173, 3],
 35364: [173, 3],
 37907: [173, 3],
 40152: [173, 3],
 42133: [173, 3],
 44483: [173, 3]}

In [49]:
len(wierd_packet)

23

In [50]:
#for debug
for i in range(0, len(evcc_send_time_anchor)):
    print(evcc_send_time_anchor[i][len(evcc_send_time_anchor[i])-1]-evcc_send_time_anchor[i][0])

1.557468
1.295960000000001
1.1256950000000003
1.086753999999999
1.168649000000002
1.0659260000000046
1.0660709999999938
1.0885199999999884
1.1038940000000252
0.9394800000000032
1.0671429999999873
1.0335599999999943
1.0483949999999993
1.0529879999999707
1.0601419999999848
1.0436919999999645
1.0686239999999998
0.9308469999999716
0.9422500000000014
0.932585999999958


In [56]:
#Processing secc send
#Append missing secc discovery response, time = 31.985675, 205.039775
with open('csv/DataLink/no_inter/no_inter_secc_send_append_missing_seccdisres.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    timelist_secc_send = []
    seq_num_secc_send = []
    data_length_secc_send = []
    info_secc_send = []
    frame_num_secc_send = []
    
    for row in rows:
        timelist_secc_send.append(float(row['Time']))
        seq_num_secc_send.append(int(row['Sequence number']))
        data_length_secc_send.append(int(row['Data Length']))
        info_secc_send.append(str(row['Info']))
        frame_num_secc_send.append(int(row['No.']))

In [57]:
#for debug
if(len(timelist_secc_send) != len(seq_num_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(data_length_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(info_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(frame_num_secc_send)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_secc_send))

1089


In [58]:
timelist_secc_send_split = []
seq_num_secc_send_split = []
data_length_secc_send_split = []
info_secc_send_split = []
frame_num_secc_send_split = []

round_start_point = 0
round_end_point = 0

sdpres_index_list = []
#Fetch first packet index of data length = 92 (SDP response) in each round
for i in range(0, len(data_length_secc_send)):
    if(i == 0 and data_length_secc_send[i] == 92):
        sdpres_index_list.append(i)
    if(i > 0 and data_length_secc_send[i] == 92 and data_length_secc_send[i-1] != 92):
        sdpres_index_list.append(i)
sdpres_index_list.append(len(data_length_secc_send)) #Add the end of list index

for i in range(0, len(sdpres_index_list)-1):
        timelist_secc_send_split.append([timelist_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        seq_num_secc_send_split.append([seq_num_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        data_length_secc_send_split.append([data_length_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        info_secc_send_split.append([info_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        frame_num_secc_send_split.append([frame_num_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])

In [59]:
#for debug
if(len(timelist_secc_send_split) != len(seq_num_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(data_length_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(info_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(frame_num_secc_send_split)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_secc_send_split)) #Output # of testing rounds

20


In [60]:
for i in range(0, len(timelist_secc_send_split)):
    print('i = ' + str(i))
    print(len(timelist_secc_send_split[i]))

i = 0
85
i = 1
56
i = 2
50
i = 3
50
i = 4
54
i = 5
50
i = 6
50
i = 7
50
i = 8
86
i = 9
50
i = 10
52
i = 11
47
i = 12
50
i = 13
52
i = 14
54
i = 15
51
i = 16
51
i = 17
48
i = 18
52
i = 19
51


In [61]:
#for debug, this is ideally the time of each testing round
for i in range(1, len(timelist_secc_send_split)):
    print(timelist_secc_send_split[i][0]-timelist_secc_send_split[i-1][0])

19.965220000000002
19.358271000000002
19.244956000000002
19.148799999999994
19.249673
19.154365
19.253039
19.245036999999996
19.244343000000015
19.15561599999998
19.167461000000003
19.123361000000017
19.25717099999997
19.250880999999993
19.251524000000018
19.148281999999995
19.245352000000025
19.05190399999998
19.046271999999988


In [62]:
for i in range(0, len(timelist_secc_send_split)):
    print('i = ' + str(i) + ' ', end = '')
    print(len(timelist_secc_send_split[i]))

i = 0 85
i = 1 56
i = 2 50
i = 3 50
i = 4 54
i = 5 50
i = 6 50
i = 7 50
i = 8 86
i = 9 50
i = 10 52
i = 11 47
i = 12 50
i = 13 52
i = 14 54
i = 15 51
i = 16 51
i = 17 48
i = 18 52
i = 19 51


In [63]:
for i in range(1, len(timelist_secc_send_split)):
    print(timelist_secc_send_split[i][0]-timelist_secc_send_split[i-1][0])

19.965220000000002
19.358271000000002
19.244956000000002
19.148799999999994
19.249673
19.154365
19.253039
19.245036999999996
19.244343000000015
19.15561599999998
19.167461000000003
19.123361000000017
19.25717099999997
19.250880999999993
19.251524000000018
19.148281999999995
19.245352000000025
19.05190399999998
19.046271999999988


In [ ]:
#Set secc time anchor in each round per stage
secc_send_time_anchor = []
#data_length_in_each_stage = [92, 173, 157, 189, 253, 173, 189, 173, 493, 173]
wierd_packet_secc = {}
#packet_loss_testing_round_5_flag = False

def printinfo(stageNum, frameNum, info_string):
    print("-----------------------------------------")
    print("Stage " + str(stageNum) + " finish. No. " + str(frameNum) + ", Info " + str(info_string))

def print_time_difference(secc_send_time_anchor_each_round):
    print("Time difference: " + str(secc_send_time_anchor_each_round[len(secc_send_time_anchor_each_round)-1]-
                          secc_send_time_anchor_each_round[len(secc_send_time_anchor_each_round)-2]))
    
def checkstage_num(stageNum, valid_stageNum, info_string, time_string, datalen, seqnum, frameNum):
    if(stageNum == valid_stageNum):
        check_wield_packet_list(frameNum)
        return True
    else:
        #debug
        print("\nWhat is this packet? Data Length = " + str(datalen) + " No. " + str(frameNum))
        print("Time " + str(time_string) + " #Seq " + str(seqnum) + " Info " + str(info_string) + "\n")
        wierd_packet_secc[frameNum] = [datalen, stageNum]
        return False

def check_wield_packet_list(frameNum):
    if(wierd_packet_secc.get(frameNum) != None):
        del wierd_packet_secc[frameNum]
        print("\nNo. " + str(frameNum) + " has been found. Delete from wierd list.\n")
    
for i in range(0, len(data_length_secc_send_split)):
#for i in [12]:
    print("\n-------- Test Round " + str(i+1) + " --------")
    #Stage 1
    secc_send_time_anchor_each_round = [timelist_secc_send_split[i][0]]
    stageNum = 1
    printinfo(stageNum, frame_num_secc_send_split[i][0], info_secc_send_split[i][0])
    stageNum = stageNum + 1
    
    for j in range(1, len(data_length_secc_send_split[i])):
        #Stage 2
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 2, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 3
        if(data_length_secc_send_split[i][j] == 157 and data_length_secc_send_split[i][j-1] != 157):
            if(checkstage_num(stageNum, 3, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 4
        if(data_length_secc_send_split[i][j] == 189 and data_length_secc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 4, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
            
                #Deal with packet loss......
                if(i == 6 and packet_loss_testing_round_6_flag == False):
                    secc_send_time_anchor_each_round.append(233.058599)
                    printinfo(stageNum, 27570, "Packet Loss")
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_6_flag = True
                
                    
                continue
                
        #Stage 5
        if(data_length_secc_send_split[i][j] == 253 and data_length_secc_send_split[i][j-1] != 253):
            if(checkstage_num(stageNum, 5, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 6
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 6, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 7
        if(data_length_secc_send_split[i][j] == 189 and data_length_secc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 7, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 8
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 8, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 9
        if(data_length_secc_send_split[i][j] == 493 and data_length_secc_send_split[i][j-1] != 493):
            if(checkstage_num(stageNum, 9, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 10
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 10, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        
        if(stageNum == 11): break
        
    secc_send_time_anchor.append(secc_send_time_anchor_each_round)

In [65]:
#for debug
print(str(len(secc_send_time_anchor)) + '\n')
for i in range(0, len(secc_send_time_anchor)):
    print(str(i) + ": " + str(len(secc_send_time_anchor[i])))

20

0: 10
1: 10
2: 10
3: 10
4: 10
5: 10
6: 10
7: 10
8: 10
9: 10
10: 10
11: 10
12: 10
13: 10
14: 10
15: 10
16: 10
17: 10
18: 10
19: 10


In [66]:
#for debug
for i in range(0, len(secc_send_time_anchor)):
    print(secc_send_time_anchor[i][len(secc_send_time_anchor[i])-1]-secc_send_time_anchor[i][0])

1.5407150000000005
1.229686000000001
1.1033029999999968
0.9916499999999928
1.063964999999996
1.0641009999999937
1.0574569999999994
1.078643999999997
1.1034679999999923
0.9417799999999943
1.062076000000019
1.0099529999999959
1.0492149999999754
1.0496070000000373
1.059784000000036
1.0390280000000303
1.06300200000004
0.9354129999999827
0.9388789999999858
0.9287740000000326


In [ ]:
'''
#Test Round 3: evcc send Too many lost packets... but no timeout
#Test Round 13: secc send Too many lost packets...
del evcc_send_time_anchor[12]
del secc_send_time_anchor[12]
del evcc_send_time_anchor[2]
del secc_send_time_anchor[2]

#for debug
print(str(len(evcc_send_time_anchor)) + '\n')
for i in range(0, len(evcc_send_time_anchor)):
    print(len(evcc_send_time_anchor[i]))
    
    
#for debug
print(str(len(secc_send_time_anchor)) + '\n')
for i in range(0, len(secc_send_time_anchor)):
    print(len(secc_send_time_anchor[i]))
'''

In [ ]:
#BLOCK ACK processing
#Time evcc sends blockACK after secc send
evcc_blockAck_time_anchor = []

blockack_index = 0
for i in range(0, len(secc_send_time_anchor)):
    evcc_blockAck_time_anchor_in_each_round = []
    
    for j in range(0, len(secc_send_time_anchor[i])):
        while(timelist_block_ack[blockack_index] < secc_send_time_anchor[i][j]):
            blockack_index = blockack_index + 1
        evcc_blockAck_time_anchor_in_each_round.append(timelist_block_ack[blockack_index])
        print("ACK time difference = " + str(
        timelist_block_ack[blockack_index] - secc_send_time_anchor[i][j]))
    
    evcc_blockAck_time_anchor.append(evcc_blockAck_time_anchor_in_each_round)

In [68]:
#SECC Packet Loss dict
#secc_packet_loss_dict[i] = {stageNum}
# i = 5, j = 0 is packet loss of evcc BLOCK ACK
# i = 3, j = 9 is packet loss of evcc BLOCK ACK
secc_packet_loss_dict = {}

#secc_packet_loss_dict[3] = {9}
#secc_packet_loss_dict[5] = {0, 1, 2, 3, 4}

#for i in secc_packet_loss_dict.get(5): print(i)
#7 in secc_packet_loss_dict.get(5)

In [69]:
#Calculate the avg. block ack latency
sum_latency = 0
cnt = 0
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) == None):
            sum_latency = sum_latency + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
            cnt = cnt + 1
        elif(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)): continue
            else:
                sum_latency = sum_latency + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
                cnt = cnt + 1
        else: continue
avg_blockAck_latency = sum_latency/cnt
print("Avg Block Ack latency = " + str(avg_blockAck_latency) + " s")
print(cnt)

Avg Block Ack latency = 0.0015511850000005812 s
200


In [ ]:
#For curiousity
#Calculate the avg. block ack latency of the appending packet loss packets
sum_latency_2 = 0
cnt = 0
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)):
                sum_latency_2 = sum_latency_2 + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
                cnt = cnt + 1
        else: continue
avg_blockAck_latency_2 = sum_latency_2/cnt
print("Avg Block Ack latency = " + str(avg_blockAck_latency_2) + " s")
print(cnt)

In [118]:
#For debug. Can see testing round 6 (i = 5) has longer block ack latency due to packet loss.
# I only append the secc send time, but didn't append the block time.
from statistics import mean

for i in range(0, len(evcc_blockAck_time_anchor)):
    print("i = " + str(i) + ": " + str(mean(evcc_blockAck_time_anchor[i])-mean(secc_send_time_anchor[i])))

i = 0: 0.09508749999999822
i = 1: 0.0808548999999914
i = 2: 0.06088309999999808
i = 3: 0.07692359999998644
i = 4: 0.09267669999999839
i = 5: 0.09291559999999777
i = 6: 0.06589310000001092
i = 7: 0.04958360000000539
i = 8: 0.09640669999998863
i = 9: 0.1306022999999641
i = 10: 0.10185220000005302
i = 11: 0.0183255999999119
i = 12: 0.05684280000002673
i = 13: 0.08050419999995029
i = 14: 0.11232169999993857
i = 15: 0.09704859999999371
i = 16: 0.058514800000011746
i = 17: 0.11826199999995879


In [119]:
#For debug
for j in range(0, 10):
    print(evcc_blockAck_time_anchor[0][j]-secc_send_time_anchor[0][j])

0.0018180000000000973
0.015291000000004829
0.15397299999999348
0.03920000000000101
0.02586799999999556
0.1058659999999989
0.2368799999999993
0.12430899999999667
0.027625000000000455
0.22004499999999894


In [120]:
#Use the avg. block ack latency for the lost secc sent packets
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)):
                evcc_blockAck_time_anchor[i][j] = secc_send_time_anchor[i][j] + avg_blockAck_latency
                print("i = " + str(i))
                print("j = " + str(j))
        else: continue

In [70]:
#for debug check wierd value
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_send_time_anchor[i][j] - evcc_send_time_anchor[i][j] < 0):
            print("i = " + str(i) + " j = " + str(j))
        if(evcc_blockAck_time_anchor[i][j] - evcc_send_time_anchor[i][j] < 0):
            print("i = " + str(i) + " j = " + str(j))

In [145]:
#For debug
#secc send is based on 802.11 capture file (not 802.11_TCP combined file)
i = 15
for j in [9]:
    print("Stage " + str(j+1))
    print(evcc_send_time_anchor[i][j])
    print(secc_send_time_anchor[i][j])
    print(evcc_blockAck_time_anchor[i][j])
    print(evcc_blockAck_time_anchor[i][j]-evcc_send_time_anchor[i][j])
    print("\n")

Stage 10
889.40234
889.592077
889.744209
0.3418689999999742




In [71]:
path = 'time_record/no_inter_80211_processing.txt'
f = open(path, 'w')
for i in range(0, len(evcc_send_time_anchor)):
    for j in range(0, len(evcc_send_time_anchor[i])):
        print((evcc_blockAck_time_anchor[i][j] - evcc_send_time_anchor[i][j])*1000, file=f, end="")
        f.writelines('\t')
    f.writelines('\t')
    print((evcc_blockAck_time_anchor[i][1] - evcc_send_time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    print((evcc_blockAck_time_anchor[i][len(evcc_blockAck_time_anchor[i])-1] - evcc_send_time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    f.writelines('20')
    f.writelines('\t')
    f.writelines('0')
    f.writelines('\t')
    f.writelines('\n')
f.close()